# Vlasov Dispersion Relation Without Landau Damping (Kappa Distribution)

In [2]:
import numpy as np
from prettytable import PrettyTable
import os
import matplotlib.pyplot as plt
import plasmapy
import scipy.signal
from scipy import special as sp
import scipy.optimize as op
from scipy import integrate
import quadpy
%config InlineBackend.figure_format = 'svg'

In [235]:
k = 0.2
L = 2*np.pi/k
T = 1  # 温度
kappa = 5.0

vmax = 5. # 速度最大值
q = 1.0
v_th = np.sqrt(2*T)
# 等离子体参量
n = 1
l_D = np.sqrt( T / n / q**2 ) # $\lambda_D^2$
w_pe = np.sqrt( n * q**2 )
print('k*l_D = ',k*l_D)
print('w_pe  = ',w_pe)

k*l_D =  0.2
w_pe  =  1.0


In [345]:
def dispersion_function(zeta,kappa):
    fac = 1j * (kappa+0.5) * (kappa-0.5) / kappa**1.5 / (kappa+1)
    z = 0.5*(1+1j*zeta/np.sqrt(kappa))
    hyp = sp.hyp2f1(1,2*kappa+2,kappa+2,z)
    return fac*hyp

# def dispersion_function(zeta,kappa):
#     r = 0.0
#     A_kappa = (kappa)**(-1.5) * sp.gamma(kappa+1) / sp.gamma(kappa-0.5)
#     def integrand(xi):
#         return  (1+xi**2/kappa)**(-kappa-1)
#     x = 0.1
#     s_p = zeta.real + x
#     s_n = zeta.real - x
#     real1 = (1/np.sqrt(np.pi)) * integrate.quad(integrand,s_n,s_p, weight='cauchy',wvar=zeta.real)[0]
#     real2 = (1/np.sqrt(np.pi)) * quadpy.quad(lambda xi: integrand(xi)/(xi-zeta),-np.inf,s_n)[0]
#     real3 = (1/np.sqrt(np.pi)) * quadpy.quad(lambda xi: integrand(xi)/(xi-zeta),s_p,np.inf)[0]
#     real = real1+real2+real3
#     return real * A_kappa
def PDRK(w,k):
    wr,wi=w
    wc=complex(wr,wi)
    zeta = wc / k /v_th_kappa
    r = 1 + (kappa/(kappa-1.5)) / k**2 / l_D**2 * ( 1 - 1/2/kappa + zeta*dispersion_function(zeta,kappa) )
    return [r.real,r.imag]

def VlasovPDRK(w,k):
    v_th_kappa = np.sqrt(2-3/kappa) * np.sqrt(T)
    zeta = w / k /v_th_kappa
    r = 1 + (kappa/(kappa-1.5)) * 1 / k**2 / l_D**2 * ( 1 - 1/2/kappa + zeta*dispersion_function(zeta,kappa).real )
    return r

def VlasovPDR(w,k):
    zeta = w / k /v_th
    r = 1 + 1 / k**2 / l_D**2 * ( 1 + zeta*plasmapy.dispersion.plasma_dispersion_func(zeta))
    return r.real

In [338]:
def SolveVlasovPDRK(k,w_init):
    s=op.root(lambda w:VlasovPDRK(w,k),w_init)
    return s
def SolvePDRK(k,wr_init,wi_init):
    s=op.root(lambda w:PDRK(w,k),[wr_init,wi_init])
    return s
def SolveVlasovPDR(k,w_init):
    s=op.root(lambda w:VlasovPDR(w,k),w_init)
    return s

In [339]:
w_up_list = np.array([])
w_down_list = np.array([])
k_list = np.linspace(0.1,0.52,num=40)
print('k = ',k_list)

k =  [0.1        0.11076923 0.12153846 0.13230769 0.14307692 0.15384615
 0.16461538 0.17538462 0.18615385 0.19692308 0.20769231 0.21846154
 0.22923077 0.24       0.25076923 0.26153846 0.27230769 0.28307692
 0.29384615 0.30461538 0.31538462 0.32615385 0.33692308 0.34769231
 0.35846154 0.36923077 0.38       0.39076923 0.40153846 0.41230769
 0.42307692 0.43384615 0.44461538 0.45538462 0.46615385 0.47692308
 0.48769231 0.49846154 0.50923077 0.52      ]


In [340]:
w_list = np.linspace(0.05,1.0,num=200)

In [350]:
kappa = 2.2
k = 0.4
wr = np.array([])
wi = np.array([])
for w in w_list:
    s = SolvePDRK(k,w,-1.0)
    if s.success==True:
        w = abs(round(s.x[0],6))
        r = abs(round(s.x[1],6))
        if w not in wr:
            wr=np.append(wr,w)
            wi=np.append(wi,r)

In [351]:
v_th_kappa = np.sqrt(2-3/kappa) * np.sqrt(T)
print(wr)
print(wi)
print(wr/k)
print(v_th)
print(v_th_kappa)

[0.112301 1.145588]
[1.096009 0.113387]
[0.2807525 2.86397  ]
1.4142135623730951
0.7977240352174657


In [343]:
plt.plot(w_list,VlasovPDRK(w_list,k))
plt.axhline(0,linestyle='--',color='black')
plt.axvline(v_th*k,linestyle='--',color='red')
plt.axvline(v_th_kappa*k,linestyle=':',color='red')
plt.axvline(k*np.sqrt(kappa)*v_th_kappa)
plt.ylim(-2,1)
plt.show()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()